<a href="https://colab.research.google.com/github/tutinz/prova4/blob/main/prova1data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creazione Dataset
per ora solo indicatori finanziari da yfinance

In [1]:
pip install yfinance

In [2]:
url = 'https://anaconda.org/conda-forge/libta-lib/0.4.0/download/linux-64/libta-lib-0.4.0-h166bdaf_1.tar.bz2'
!curl -L $url | tar xj -C /usr/lib/x86_64-linux-gnu/ lib --strip-components=1
url = 'https://anaconda.org/conda-forge/ta-lib/0.4.19/download/linux-64/ta-lib-0.4.19-py310hde88566_4.tar.bz2'
!curl -L $url | tar xj -C /usr/local/lib/python3.10/dist-packages/ lib/python3.10/site-packages/talib --strip-components=3
import talib

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4469    0  4469    0     0  12964      0 --:--:-- --:--:-- --:--:-- 12991
100  517k  100  517k    0     0   628k      0 --:--:-- --:--:-- --:--:-- 3272k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4497    0  4497    0     0  16762      0 --:--:-- --:--:-- --:--:-- 16779
100  392k  100  392k    0     0   535k      0 --:--:-- --:--:-- --:--:--  535k


In [4]:
import yfinance as yf
BTC_Ticker = yf.Ticker("BTC-USD")
BTC_Data = BTC_Ticker.history(period="max")
BTC_Data

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2014-09-17 00:00:00+00:00,465.864014,468.174011,452.421997,457.334015,21056800,0.0,0.0
2014-09-18 00:00:00+00:00,456.859985,456.859985,413.104004,424.440002,34483200,0.0,0.0
2014-09-19 00:00:00+00:00,424.102997,427.834991,384.532013,394.795990,37919700,0.0,0.0
2014-09-20 00:00:00+00:00,394.673004,423.295990,389.882996,408.903992,36863600,0.0,0.0
2014-09-21 00:00:00+00:00,408.084991,412.425995,393.181000,398.821014,26580100,0.0,0.0
...,...,...,...,...,...,...,...
2023-11-18 00:00:00+00:00,36625.371094,36839.281250,36233.312500,36585.703125,11886022717,0.0,0.0
2023-11-19 00:00:00+00:00,36585.765625,37509.355469,36414.597656,37386.546875,12915986553,0.0,0.0
2023-11-20 00:00:00+00:00,37374.074219,37756.820312,36882.531250,37476.957031,20888209068,0.0,0.0


Comincio con i soli indicatori finanziari come features del modello

In [ ]:
def fetch_stock_data(symbol, start_date, end_date):
    stock_data = yf.download(symbol, start=start_date, end=end_date)
    return stock_data

def calculate_technical_indicators(stock_data):
    # Calculate technical indicators using ta-lib
    stock_data['SMA_50'] = talib.SMA(stock_data['Close'], timeperiod=50)
    stock_data['SMA_200'] = talib.SMA(stock_data['Close'], timeperiod=200)
    stock_data['RSI'] = talib.RSI(stock_data['Close'])
    # Add more indicators as needed
    stock_data['SMA'] = talib.SMA(stock_data['Close'], timeperiod=14)
    stock_data['EMA'] = talib.EMA(stock_data['Close'], timeperiod=14)
    stock_data['RSI'] = talib.RSI(stock_data['Close'], timeperiod=14)
    stock_data['MACD'], _, _ = talib.MACD(stock_data['Close'])
    stock_data['BollingerBandsUpper'], stock_data['BollingerBandsMiddle'], stock_data['BollingerBandsLower'] = talib.BBANDS(stock_data['Close'])
    stock_data['ADX'] = talib.ADX(stock_data['High'], stock_data['Low'], stock_data['Close'], timeperiod=14)
    stock_data['ATR'] = talib.ATR(stock_data['High'], stock_data['Low'], stock_data['Close'], timeperiod=14)
    stock_data['MOM'] = talib.MOM(stock_data['Close'], timeperiod=10)
    stock_data['CCI'] = talib.CCI(stock_data['High'], stock_data['Low'], stock_data['Close'], timeperiod=14)
    stock_data['OBV'] = talib.OBV(stock_data['Close'], stock_data['Volume'])

    return stock_data

def save_to_csv(stock_data, csv_filename):
    stock_data.to_csv(csv_filename, index=True)

if __name__ == "__main__":
    # Replace 'AAPL' with the desired stock symbol
    symbol = 'BTC-USD'
    start_date = '2020-01-01'
    end_date = '2023-01-01'
    csv_filename = 'financial_indicators.csv'

    # Fetch historical stock data
    stock_data = fetch_stock_data(symbol, start_date, end_date)

    # Calculate technical indicators
    stock_data_with_indicators = calculate_technical_indicators(stock_data)

    # Save data to CSV
    save_to_csv(stock_data_with_indicators, csv_filename)


[*********************100%%**********************]  1 of 1 completed


# devo modificare il csv qui togliendo le righe in cui i dati degli indicatori sono missing..   parto dal 200, per via della sma(200)!!!!

In [ ]:
import pandas as pd

# Load the CSV file into a DataFrame
input_file_path = 'financial_indicators.csv'
output_file_path = 'financial_indicators1.csv'

df = pd.read_csv(input_file_path)

# Use the drop method to remove the identified row(s)... con ciclo for????
for i in range(205):
#    print(i)
  df = df.drop(i)

# Write the updated DataFrame to a new CSV file
df.to_csv(output_file_path, index=False)

# Modello Random Forests

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# Load the data
data = pd.read_csv('financial_indicators1.csv')
# Assuming 'Close' is the target variable (price)
target = data['Close']
features = data.drop(['Date', 'Close'], axis=1)  # Assuming 'Date' is not a feature
data

Questo split non mi convince... non penso che vada fatto random altrimenti si perde sequenzialità temporale... forse è questo il motivo del grande errore...

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)
X_train

Bisognerà fare preprocessing?? e feature selection ma prima voglio aggiungere dati on-chain... o forse poteri farlo anche ora...

In [ ]:
# Create a Random Forest Regressor
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
rf_model.fit(X_train, y_train)

# Make predictions
predictions = rf_model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, predictions)
print(f'Mean Squared Error: {mse}')

Mean Squared Error: 25928.624026020374


L'errore finale viene grandissimo!!!

Errore megagalattico... devo capire come interpretare bene questo indicatore... era che non avevo fatto un giusto split e neanche preprocessing!!!

#Rifaccio con split train test ragionevole e Preprocessing!!!

In [ ]:
# Load the data
data = pd.read_csv('financial_indicators1.csv')
# Assuming 'Close' is the target variable (price)
#target = data['Close']
#features = data.drop(['Date', 'Close'], axis=1)  # Assuming 'Date' is not a feature
#data

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer

# Load the CSV dataset
#file_path = 'financial_indicators1.csv'  # Replace with the actual path to your CSV file
df = data.drop(['Date'], axis=1)

# Display the initial dataset
print("Initial Dataset:")
print(df.head())

# Handling missing values (you can customize this based on your needs)
imputer = SimpleImputer(strategy='mean')  # Other strategies: median, most_frequent
df_imputed = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

# Display the dataset after handling missing values
print("\nDataset after handling missing values:")
print(df_imputed.head())

# Normalizing numerical features using Min-Max scaling
scaler_minmax = MinMaxScaler()
df_normalized = pd.DataFrame(scaler_minmax.fit_transform(df_imputed), columns=df_imputed.columns)

# Display the dataset after normalization
print("\nDataset after normalization:")
print(df_normalized.head())

# Standardizing numerical features using z-score (standardization)
scaler_standard = StandardScaler()
df_standardized = pd.DataFrame(scaler_standard.fit_transform(df_imputed), columns=df_imputed.columns)

# Display the dataset after standardization
print("\nDataset after standardization:")
print(df_standardized.head())


In [ ]:
df_standardized['Date'] = data['Date']
#df_standardized

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

# Load your time series dataset (replace 'your_dataset.csv' with the actual file path)
dataset = df_standardized

# Assuming your dataset has a 'Date' column, convert it to a datetime object
dataset['Date'] = pd.to_datetime(dataset['Date'])

# Sort the dataset by date
dataset.sort_values(by='Date', inplace=True)

# Specify the number of splits for time series cross-validation
n_splits = 2  # You can adjust this value based on your requirements

# Initialize TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=n_splits)

# Split the dataset into training and testing sets
for train_index, test_index in tscv.split(dataset):
    train_set = dataset.iloc[train_index]
    test_set = dataset.iloc[test_index]

    # Your training and testing operations here
    # For example, print the size of each split
    print("Train set size:", len(train_set))
    print("Test set size:", len(test_set))
    print("---")

Train set size: 297
Test set size: 297
---
Train set size: 594
Test set size: 297
---


In [ ]:
# Assuming 'Close' is the target variable (price)
y_train = train_set['Close']
X_train = train_set.drop(['Date', 'Close'], axis=1)  # Assuming 'Date' is not a feature
y_test = test_set['Close']
X_test = test_set.drop(['Date', 'Close'], axis=1)  # Assuming 'Date' is not a feature

In [ ]:
# Create a Random Forest Regressor
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
rf_model.fit(X_train, y_train)

# Make predictions
predictions = rf_model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, predictions)
print(f'Mean Squared Error: {mse}')

Mean Squared Error: 0.25724022346368713


In [ ]:
# Predict on new data
# Assuming 'new_data' is a DataFrame with the same structure as your training data
new_data = pd.read_csv('new_data.csv')
new_predictions = rf_model.predict(new_data)

print('Predictions for new data:')
print(new_predictions)

# Classificatore
Sempre con Random Forests

Anche qui c'è da sistemare: preprocessing e split

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Load the data
data = pd.read_csv('financial_indicators1.csv')
# Assuming 'Close' is the target variable (price)
# Create a new column 'Target' representing the binary classification task (1: price increase, 0: price decrease or same)
data['Target'] = (data['Close'].shift(-1) > data['Close']).astype(int)

# Drop the last row since the target for it is not available
data = data.dropna()
df_standardized = df_standardized.dropna()
df_standardized['Target'] = data['Target']
df_standardized = df_standardized.drop(['Close'], axis=1)
df_standardized

,Open,High,Low,Adj Close,Volume,SMA_50,SMA_200,RSI,SMA,EMA,...,BollingerBandsUpper,BollingerBandsMiddle,BollingerBandsLower,ADX,ATR,MOM,CCI,OBV,Date,Target
0,-1.538912,-1.549487,-1.532555,-1.543845,-1.048627,-1.572071,-1.681669,0.661912,-1.564118,-1.568852,...,-1.556255,-1.552047,-1.537711,-1.202499,-1.495133,0.046800,1.366129,-2.507199,2020-07-24,1
1,-1.541865,-1.544412,-1.529326,-1.534842,-1.045771,-1.572056,-1.681136,0.988944,-1.562105,-1.565926,...,-1.556691,-1.545449,-1.523195,-1.066335,-1.494480,0.089283,1.300727,-2.464733,2020-07-25,1
2,-1.532837,-1.524464,-1.521205,-1.520200,-0.851505,-1.571727,-1.680493,1.408743,-1.559208,-1.561418,...,-1.545134,-1.538628,-1.521773,-0.832036,-1.481002,0.153625,1.509928,-2.412302,2020-07-26,1
3,-1.518407,-1.444830,-1.504576,-1.450491,-0.111321,-1.570114,-1.679396,2.363127,-1.551156,-1.548122,...,-1.478995,-1.519775,-1.556544,-0.461191,-1.401489,0.391350,2.449770,-2.321900,2020-07-27,0
4,-1.447069,-1.450697,-1.456323,-1.455502,-0.439914,-1.568620,-1.678429,2.152659,-1.543461,-1.537274,...,-1.453194,-1.502643,-1.549328,-0.116836,-1.381510,0.372249,1.854318,-2.395445,2020-07-28,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,-1.068535,-1.091056,-1.058372,-1.082828,-1.088706,-1.081805,-0.884717,-0.480355,-1.073336,-1.079954,...,-1.120370,-1.076826,-1.020910,-0.998545,-1.345282,-0.036027,-0.472355,-1.477038,2022-12-27,0
887,-1.081548,-1.103033,-1.067942,-1.093396,-1.026053,-1.084408,-0.888877,-0.676040,-1.079157,-1.082757,...,-1.115755,-1.079970,-1.032813,-0.957006,-1.349030,-0.064443,-1.178794,-1.520515,2022-12-28,1
888,-1.092072,-1.110307,-1.067205,-1.087632,-1.152316,-1.083411,-0.892442,-0.532892,-1.082486,-1.084410,...,-1.117847,-1.082612,-1.036062,-0.918434,-1.360861,0.026513,-1.267932,-1.483515,2022-12-29,0
889,-1.086363,-1.110827,-1.073841,-1.090185,-1.079706,-1.084699,-0.894515,-0.584095,-1.082693,-1.086187,...,-1.121248,-1.085692,-1.038760,-0.860219,-1.365831,-0.086357,-1.278818,-1.524240,2022-12-30,0


In [ ]:
from sklearn.model_selection import TimeSeriesSplit

# Load your time series dataset (replace 'your_dataset.csv' with the actual file path)
dataset = df_standardized

# Assuming your dataset has a 'Date' column, convert it to a datetime object
dataset['Date'] = pd.to_datetime(dataset['Date'])

# Sort the dataset by date
dataset.sort_values(by='Date', inplace=True)

# Specify the number of splits for time series cross-validation
n_splits = 2  # You can adjust this value based on your requirements

# Initialize TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=n_splits)

# Split the dataset into training and testing sets
for train_index, test_index in tscv.split(dataset):
    train_set = dataset.iloc[train_index]
    test_set = dataset.iloc[test_index]

    # Your training and testing operations here
    # For example, print the size of each split
    print("Train set size:", len(train_set))
    print("Test set size:", len(test_set))
    print("---")

In [ ]:
# Assuming 'Close' is the target variable (price)
y_train = train_set['Target']
X_train = train_set.drop(['Date', 'Target'], axis=1)  # Assuming 'Date' is not a feature
y_test = test_set['Target']
X_test = test_set.drop(['Date', 'Target'], axis=1)  # Assuming 'Date' is not a feature

Rivedere metrica accuracy...ok

In [ ]:
# Create a Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
rf_classifier.fit(X_train, y_train)

# Make predictions
predictions = rf_classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
print(f'Accuracy: {accuracy}')

print('Classification Report:')
print(classification_report(y_test, predictions))

Accuracy: 0.4612794612794613
Classification Report:
              precision    recall  f1-score   support

           0       0.50      0.13      0.21       160
           1       0.45      0.85      0.59       137

    accuracy                           0.46       297
   macro avg       0.48      0.49      0.40       297
weighted avg       0.48      0.46      0.39       297



L'accuratezza non mi sembra soddisfacente...